# LN fit to deep-retina  

When you fit an LN model to the responses of deep-retina, what does it look like? How does this LN approximation to the convolutional neural network compare to LN models learned directly from the data?

In [94]:
from __future__ import absolute_import
import numpy as np
import pickle
from scipy.io import loadmat
import os.path as path
import matplotlib
import h5py
#Force matplotlib to not use any Xwindows else will crash on Rye
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# from matplotlib.pyplot import *
from scipy.optimize import curve_fit
from scipy.stats import pearsonr
# Keras imports
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.core import Dense, Dropout, Activation, TimeDistributedDense, Reshape, Permute
from keras.layers.recurrent import LSTM, GRU, JZS1, JZS2, JZS3
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.layers.embeddings import Embedding
from keras.regularizers import l1, l2, activity_l1, activity_l2
from keras.callbacks import Callback
from keras.objectives import poisson_loss
import theano
import theano.tensor as T
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
data_dir_naturalscenes = '/Users/lmcintosh/Documents/Stanford/00 Baccus Lab/Data 2015_10_07/naturalscene.h5'
data_dir_whitenoise = '/Users/lmcintosh/Documents/Stanford/00 Baccus Lab/Data 2015_10_07/whitenoise.h5'
weights_dir = '/Users/lmcintosh/Git/deep-retina/trained_weights_epoch0_batch200.h5'

In [72]:
weights = h5py.File(weights_dir, 'r')

In [73]:
W0 = weights['layer_0']['param_0']
W0.shape

(16, 40, 4, 4)

In [95]:
from pyret.visualizations import plotsta
import pyret.filtertools as ft

In [75]:
plotsta(np.linspace(0,400,40), W0[0].T)

ValueError: operands could not be broadcast together with shapes (16,40) (16,) 

In [13]:
W0[0].T.shape

(4, 4, 40)

In [89]:
yoda = W0[0]
spaceblah, timeblah = ft.decompose(yoda.T)

ValueError: operands could not be broadcast together with shapes (16,40) (16,) 

In [21]:
spaceblah.shape

(40, 4)

In [24]:
yoda.shape

(40, 4, 4)

In [26]:
yoda.T.shape

(4, 4, 40)

In [96]:
fake_sta = np.random.randn(7,7,40)

In [97]:
spaceblah, timeblah = ft.decompose(fake_sta)

In [98]:
spaceblah.shape

(7, 7)

In [99]:
timeblah.shape

(40,)

# Replicate Aran's Code

In [65]:
from __future__ import absolute_import
import numpy as np
import pickle
from scipy.io import loadmat
import os.path as path
import matplotlib
import h5py
#Force matplotlib to not use any Xwindows else will crash on Rye
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# from matplotlib.pyplot import *
from scipy.optimize import curve_fit
from scipy.stats import pearsonr
# Keras imports
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.core import Dense, Dropout, Activation, TimeDistributedDense, Reshape, Permute
from keras.layers.recurrent import LSTM, GRU, JZS1, JZS2, JZS3
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.layers.embeddings import Embedding
from keras.regularizers import l1, l2, activity_l1, activity_l2
from keras.callbacks import Callback
from keras.objectives import poisson_loss
import theano
import theano.tensor as T

from keras.layers.core import Dense, Dropout, Activation, Flatten, TimeDistributedDense
from keras.layers.convolutional import Convolution2D, MaxPooling2D

Still had to modify to remove time distributed conv and pool layers

In [67]:
init_method='normal'

model = Sequential()
reg = 0.0 #could try 0.01

model.add(Convolution2D(16, 40, 9, 9, init=init_method, border_mode='full', subsample=(1,1), W_regularizer=l1(reg))) 
model.add(Activation('relu'))

model.add(MaxPooling2D(poolsize=(2, 2), ignore_border=True))

model.add(Flatten())
model.add(Dense(13456, 32, init=init_method, W_regularizer=l1(reg)))
model.add(Activation('relu'))
model.add(Dense(32, 1, init=init_method, activation='softplus', W_regularizer=l1(reg)))
model.compile(loss='poisson_loss', optimizer='rmsprop')

WARNING (theano.sandbox.cuda.opt): Optimization Warning: Got the following error, but you can ignore it. This could cause less GpuElemwise fused together.
(4, -538971137)
(4, -538971137)
ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR:theano.gof.opt:Optimization failure due to: constant_folding
ERROR (theano.gof.opt): TRACEBACK:
ERROR:theano.gof.opt:TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Library/Frameworks/EPD64.framework/Versions/7.3/lib/python2.7/site-packages/theano/gof/opt.py", line 1493, in process_node
    replacements = lopt.transform(node)
  File "/Library/Frameworks/EPD64.framework/Versions/7.3/lib/python2.7/site-packages/theano/tensor/opt.py", line 4686, in constant_folding
    required = thunk()
  File "/Library/Frameworks/EPD64.framework/Versions/7.3/lib/python2.7/site-packages/theano/gof/op.py", line 743, in rval
    fill_storage()
  File "/Library/Frameworks/EPD64.framework/Versions/7.3/lib/python2.7/site

In [69]:
model.load_weights(weights_dir)

ValueError: DeepCopyOp: the copy failed!
Apply node that caused the error: DeepCopyOp(<CudaNdarrayType(float32, 4D)>)
Inputs types: [CudaNdarrayType(float32, 4D)]
Inputs shapes: [(16, 40, 9, 9)]
Inputs strides: [(3240, 81, 9, 1)]
Inputs values: ['not shown']

Debugprint of the apply node: 
DeepCopyOp [@A] <CudaNdarrayType(float32, 4D)> ''   
 |<CudaNdarrayType(float32, 4D)> [@B] <CudaNdarrayType(float32, 4D)>

Storage map footprint:
 - <CudaNdarrayType(float32, 4D)>, Shape: (16, 40, 9, 9), ElemSize: 4 Byte(s), TotalSize: 207360 Byte(s)

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.